In [ ]:
import pandas as pd
import numpy as np
import os
import json, requests
import time

In [ ]:
city_country = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/city_country_separate.csv')

In [ ]:
key = os.getenv('G_API_KEY')
#base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'
#maxprice = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}&query={}&minprice={}&maxprice={}'

In [ ]:
urll = maxprice.format(key, query3.format('portland'), 2, 2)

In [ ]:
query_dict = {'attractions': "{}+points+of+interest",
              'restaurants': "best+restaurants+in+{}",
             'restaurants_one': 'best+restaurants+in+{}&minprice=1&maxprice=1',
             'restaurants_two': 'best+restaurants+in+{}&minprice=2&maxprice=2',
             'restaurants_three': 'best+restaurants+in+{}&minprice=3&maxprice=3',
             'restaurants_four': 'best+restaurants+in+{}&minprice=4&maxprice=4'} #build seperate query for min/max price 
city_list = city_country.iloc[0:2, :].copy()


In [ ]:
print(list(query_dict.keys())[list(query_dict.values()).index('best+restaurants+in+{}&minprice=3&maxprice=3')])

for table_name, query in query_dict.items():
    if table_name == 'attractions':
        print('YES')
    else:
        print('NO/')

In [ ]:
query3 = query_dict['restaurants_three']
#url_price = base_url.format(key) + '&query={}'.format(query3.format('portland'))

In [ ]:
query_test = query_dict['attractions'], query_dict['restaurants_three']

In [ ]:
#utils.py
def format_city(city_list):
    city_list.replace(' ', '+', regex=True, inplace=True)
    return city_list

In [ ]:
city_list = format_city(city_list)

In [ ]:
for c in city_list.city:
    print(type(c))

In [ ]:

cc = city_list.loc[city_list['city'] == 'Hong+Kong', 'country'].item()
print(cc)

In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'

def city_query_selection(city_df, query_dict):
    for query in query_dict:
        for city in city_df.city:
            country = city_df.loc[city_df['city'] == city, 'country'].item()
            id = city_df.loc[city_df['city'] == city, 'id'].item()
            pipeline(city, query_dict[query], country, id, query_dict)

    return 


# Pipeline - main.py
def pipeline(city, query, country, id, query_dict):
    url = build_url(city, query, country)
    data = find_places_api(url)
    table_name = find_table_name(query_dict, query)
    df = create_df(data, city, country, id, table_name)
    #write_db(df)
    
    return df


def build_url(city, query, country):
    base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'
    cc = [city, country]
    cc = '+'.join(cc)
    url = base_url.format(key) + '&query={}'.format(query.format(cc))
    return url


def make_next_page_url(results, base_url):
    
    return base_url.format(key) + '&pagetoken={}'.format(results['next_page_token'])


def find_places_api(url, data=[]):
    results = requests.get(url).json()
    time.sleep(3)
    data = data + results['results']
    print(results.keys())
    print(results['status'])
    if 'next_page_token' not in results.keys() and results['status']=='OK':
        print('no next')
        return data
    elif results['status']=='OK':
        print('next')
        new_url = make_next_page_url(results, base_url=base_url)
        return find_places_api(new_url, data)
    else:
        return data 

def create_df(json_data, city, country, id, table_name):
    df = pd.json_normalize(json_data)
    df['city'] = city.replace('+', ' ')
    df['country'] = country.replace('+', ' ')
    df['id'] = id
    df = column_selection(df, table_name)
    return df


def column_selection(df, table_name):
    if table_name == 'attractions':
        cols_to_keep = ['country', 'city', 'name', 'formatted_address',
                        'rating', 'user_ratings_total', 'types',
                        'geometry.location.lat', 'geometry.location.lng', 'place_id', 'id']
    else:
        cols_to_keep = ['country', 'city', 'name', 'formatted_address', 'price_level',
                        'rating', 'user_ratings_total', 'types',
                        'geometry.location.lat', 'geometry.location.lng', 'place_id', 'id']
    
    df = df[cols_to_keep].copy()
    df.rename(columns={'formatted_address': 'address',
    'geometry.location.lat': 'latitude', 'geometry.location.lng': 'longitude'}, inplace=True)
    return df
    
    
def find_table_name(query_dict, query):
    table_name = (list(query_dict.keys())[list(query_dict.values()).index(query)])
    
    return table_name

#db_connect.py
def write_db(table_name, df):
    df.to_sql(table_name, conn, if_exists="append")
    return 


# append data or delete and recreate when rerunning
# error handling if write to db doesn't suceed report error back. 

In [ ]:
data = build_url(city_list.city[1], query3, city_list.country[1])

In [ ]:
data

In [ ]:
city_list.city[1]

In [ ]:
huh = city_query_selection(city_list, query_dict, )

In [ ]:
type(huh)

In [ ]:
c = city_list.city[0]
q = 'best+restaurants+in+{}'
country = city_list.country[0]

In [ ]:
url = build_url(c, q, country)

In [ ]:
url

In [ ]:
data = find_places_api(url)
data

In [ ]:
id = 5
df5 = create_df(data, c, country, id, 'table')


In [ ]:
print(q)

In [ ]:
df5

In [ ]:
tog = city_list.city[0]+city_list.country[0]

In [ ]:
lst = [city_list.city[0], city_list.country[0]]

In [ ]:
lst

In [ ]:
lst = '+'.join(lst)

In [ ]:
lst